## Fine-Tune Stage 1

### Linux

In [1]:
# ===== LoRA 版完整训练脚本（Linux + 4090 24GB，分层采样 + Trainer）=====
# 说明：不使用 bitsandbytes/QLoRA，避免 .to 限制；仅训练 LoRA 适配器与分类头
# 依赖：pip install transformers datasets peft accelerate scikit-learn pandas numpy evaluate

import os, random, torch, numpy as np
from pathlib import Path

# --- 基础环境 ---
os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "0")  # 明确禁用 MPS
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0")
torch.backends.cuda.matmul.allow_tf32 = True
assert torch.cuda.is_available(), "未检测到 CUDA，请确认在 Linux + NVIDIA 驱动环境运行。"

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ===== 路径与基础配置 =====
PROJECT_ROOT = Path("/media/veracrypt1/DS5500/project")
DATA_CSV     = PROJECT_ROOT / "data" / "processed" / "sentiment_df.csv"
OUT_DIR      = PROJECT_ROOT / "stage1" / "stage1_lora_sentiment"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 模型名称（若无 Llama 授权，改为 Qwen）
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

print("Device: cuda | GPU:", torch.cuda.get_device_name(0),
      "| VRAM (GB):", round(torch.cuda.get_device_properties(0).total_memory/1024**3,2))

# ===== 读数 & 分层采样 =====
import pandas as pd
df = pd.read_csv(DATA_CSV)
assert {"text","sentiment","split"}.issubset(df.columns)

def norm_split(s):
    s = str(s).strip().lower()
    if s in {"val","valid","validation","dev"}: return "validation"
    if s in {"train","training"}: return "train"
    if s in {"test","testing"}: return "test"
    return s

df["split_norm"] = df["split"].map(norm_split)

raw_train_df = df[df["split_norm"]=="train"][["text","sentiment"]].rename(columns={"sentiment":"label"}).reset_index(drop=True)
val_df       = df[df["split_norm"]=="validation"][["text","sentiment"]].rename(columns={"sentiment":"label"}).reset_index(drop=True)

# 转 int，避免类型警告
raw_train_df["label"] = raw_train_df["label"].astype(int)
val_df["label"]       = val_df["label"].astype(int)

print("\n=== 原始训练/验证集规模 ===")
print("Train:", raw_train_df.shape, "Validation:", val_df.shape)
print("原始训练集标签分布：\n", raw_train_df["label"].value_counts().sort_index())

# 分层采样（默认：下采样到最小类；要上采样把 strategy 改为 'oversample'）
def stratified_balance(df_in: pd.DataFrame, label_col="label", strategy="undersample", random_state=SEED):
    g = df_in.groupby(label_col, group_keys=False)
    counts = g.size()
    if strategy == "undersample":
        n = counts.min()
        return g.apply(lambda x: x.sample(n=n, replace=False, random_state=random_state)).reset_index(drop=True)
    elif strategy == "oversample":
        n = counts.max()
        return g.apply(lambda x: x.sample(n=n, replace=True,  random_state=random_state)).reset_index(drop=True)
    else:
        raise ValueError("strategy 必须是 'undersample' 或 'oversample'")

train_df = stratified_balance(raw_train_df, label_col="label", strategy="undersample", random_state=SEED)
print("\n=== 分层采样后的训练集 ===")
print("Balanced Train:", train_df.shape)
print("标签分布：\n", train_df["label"].value_counts().sort_index())
print("\n验证集标签分布（未平衡）：\n", val_df["label"].value_counts().sort_index())

# ===== HF Datasets & Tokenizer =====
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df)
})
num_labels = 3  # 0=neg,1=neu,2=pos

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tok.padding_side = "right"
tok.truncation_side = "right"
if tok.pad_token_id is None and tok.eos_token_id is not None:
    tok.pad_token = tok.eos_token

MAX_LEN = 128
def tokenize_batch(batch):
    enc = tok(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
    )
    enc["labels"] = batch["label"]
    return enc

tok_ds = ds.map(tokenize_batch, batched=True, remove_columns=["text","label"])
tok_ds = tok_ds.with_format("torch")

# ===== 模型加载（LoRA，仅 bf16/FP16；不量化）=====
from transformers import AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

use_bf16   = torch.cuda.is_bf16_supported()
torch_dtype = torch.bfloat16 if use_bf16 else torch.float16

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    problem_type="single_label_classification",
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
)
model.to("cuda")  # 直接放到 GPU

# pad token & 训练配置
if tok.pad_token_id is None and tok.eos_token_id is not None:
    tok.pad_token = tok.eos_token
model.config.pad_token_id = tok.pad_token_id
model.config.use_cache = False
model.gradient_checkpointing_enable()   # 省显存

# LoRA 仅打注意力投影层（最省显存，够用）
lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    # 显存足够可加："gate_proj","up_proj","down_proj"
)
model = get_peft_model(model, lora_cfg)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTrainable params (LoRA + head): {trainable/1e6:.2f}M")

# ===== 指标 =====
from sklearn.metrics import f1_score, accuracy_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average="macro")
    return {"accuracy": float(acc), "macro_f1": float(macro_f1)}

# ===== 训练参数（24GB 稳妥配置）=====
from transformers import TrainingArguments, Trainer

BATCH_SIZE = 8    # 起步稳妥；能跑再调 6/8
GRAD_ACCUM = 4    # 提高全局 batch
MAX_STEPS  = 2000

args = TrainingArguments(
    output_dir=str(OUT_DIR),
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    # max_steps=MAX_STEPS,
    num_train_epochs=6,
    warmup_steps=100,
    evaluation_strategy="epoch",
    eval_steps=200,
    save_strategy="epoch",
    save_steps=200,
    save_total_limit = 2,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,

    logging_steps=50,
    bf16=use_bf16,           # 4090 支持 bf16；若报错则改为 False 且 fp16=True
    fp16=not use_bf16,
    dataloader_pin_memory=True,
    dataloader_num_workers=8,
    optim="adamw_torch",
    report_to="none",
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tok_ds["train"],
    eval_dataset=tok_ds["validation"],
    tokenizer=tok,
    compute_metrics=compute_metrics,
)

# ===== 训练 & 评估 =====
train_result = trainer.train()
eval_result  = trainer.evaluate()
print("\n=== Eval Result ===\n", eval_result)

# ===== 保存 LoRA 适配器 =====
adapter_dir = OUT_DIR / "adapter"
adapter_dir.mkdir(parents=True, exist_ok=True)
model.save_pretrained(adapter_dir)
tok.save_pretrained(adapter_dir)
print("Saved LoRA adapter to:", adapter_dir)

Device: cuda | GPU: NVIDIA GeForce RTX 4090 | VRAM (GB): 23.64

=== 原始训练/验证集规模 ===
Train: (254159, 2) Validation: (8101, 2)
原始训练集标签分布：
 label
0     90403
1     62297
2    101459
Name: count, dtype: int64

=== 分层采样后的训练集 ===
Balanced Train: (186891, 2)
标签分布：
 label
0    62297
1    62297
2    62297
Name: count, dtype: int64

验证集标签分布（未平衡）：
 label
0    2740
1    2098
2    3263
Name: count, dtype: int64


/tmp/ipykernel_43519/2154849521.py:61: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(lambda x: x.sample(n=n, replace=False, random_state=random_state)).reset_index(drop=True)
/home/exx/venvs/llamaFT_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 15.29it/s]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.1-8B-Instruct and are newly initialized: ['score.weight']
You should 


Trainable params (LoRA + head): 13.64M


/home/exx/venvs/llamaFT_env/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/35040 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after 

{'loss': 2.5773, 'grad_norm': 23.60845375061035, 'learning_rate': 1e-05, 'epoch': 0.01}


...

{'eval_loss': 0.5413411259651184, 'eval_accuracy': 0.7899024811751636, 'eval_macro_f1': 0.7763680389338427, 'eval_runtime': 116.1483, 'eval_samples_per_second': 69.747, 'eval_steps_per_second': 8.722, 'epoch': 6.0}


100%|██████████| 35040/35040 [15:01:40<00:00,  1.54s/it]

{'train_runtime': 54100.6937, 'train_samples_per_second': 20.727, 'train_steps_per_second': 0.648, 'train_loss': 0.41276632228942767, 'epoch': 6.0}



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The


=== Eval Result ===
 {'eval_loss': 0.4712909758090973, 'eval_accuracy': 0.7980496235032712, 'eval_macro_f1': 0.7849208035647255, 'eval_runtime': 115.9615, 'eval_samples_per_second': 69.859, 'eval_steps_per_second': 8.736, 'epoch': 5.999486345347145}
Saved LoRA adapter to: /media/veracrypt1/DS5500/project/stage1/stage1_lora_sentiment/adapter


In [1]:
# =========================
# Stage 1: LoRA fine-tune (sentiment 3-way) with well-structured saving
# =========================
import os, random, json, torch, numpy as np, pandas as pd
from pathlib import Path

# --- Environment ---
os.environ.setdefault("PYTORCH_ENABLE_MPS_FALLBACK", "0")
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0")
torch.backends.cuda.matmul.allow_tf32 = True
assert torch.cuda.is_available(), "CUDA not found. Please run on Linux + NVIDIA."

SEED = 47
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# --- Paths ---
PROJECT_ROOT = Path("/media/veracrypt1/DS5500/project")
DATA_CSV     = PROJECT_ROOT / "data" / "processed" / "sentiment_df.csv"
# HF Trainer working dir (checkpoints/logs stay here)
OUT_DIR      = PROJECT_ROOT / "stage1" / "stage1_lora_sentiment"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Structured results (what you asked)
STAGE1_RES      = PROJECT_ROOT / "stage1" / "stage1Result"
EVAL_DIR        = STAGE1_RES / "eval"
OUTPUT_DIR      = STAGE1_RES / "output"
EVAL_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# --- Base model ---
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

print("Device: cuda | GPU:", torch.cuda.get_device_name(0),
      "| VRAM (GB):", round(torch.cuda.get_device_properties(0).total_memory/1024**3, 2))

Device: cuda | GPU: NVIDIA GeForce RTX 4090 | VRAM (GB): 23.64


In [2]:
# =========================
# 1) Data & Stratified balancing
# =========================
df = pd.read_csv(DATA_CSV)
assert {"text","sentiment","split"}.issubset(df.columns)

def norm_split(s):
    s = str(s).strip().lower()
    if s in {"val","valid","validation","dev"}: return "validation"
    if s in {"train","training"}:               return "train"
    if s in {"test","testing"}:                 return "test"
    return s

df["split_norm"] = df["split"].map(norm_split)

raw_train_df = df[df["split_norm"]=="train"][["text","sentiment"]].rename(columns={"sentiment":"label"}).reset_index(drop=True)
val_df       = df[df["split_norm"]=="validation"][["text","sentiment"]].rename(columns={"sentiment":"label"}).reset_index(drop=True)

raw_train_df["label"] = raw_train_df["label"].astype(int)
val_df["label"]       = val_df["label"].astype(int)

print("\n=== Original train/val sizes ===")
print("Train:", raw_train_df.shape, "Validation:", val_df.shape)
print("Train label dist:\n", raw_train_df["label"].value_counts().sort_index())

def stratified_balance(df_in: pd.DataFrame, label_col="label", strategy="undersample", random_state=SEED):
    g = df_in.groupby(label_col, group_keys=False)
    counts = g.size()
    if strategy == "undersample":
        n = counts.min()
        return g.apply(lambda x: x.sample(n=n, replace=False, random_state=random_state)).reset_index(drop=True)
    elif strategy == "oversample":
        n = counts.max()
        return g.apply(lambda x: x.sample(n=n, replace=True,  random_state=random_state)).reset_index(drop=True)
    else:
        raise ValueError("strategy must be 'undersample' or 'oversample'")

train_df = stratified_balance(raw_train_df, label_col="label", strategy="undersample", random_state=SEED)
print("\n=== Balanced train ===")
print("Balanced Train:", train_df.shape)
print("Label dist:\n", train_df["label"].value_counts().sort_index())
print("\nValidation label dist (kept original):\n", val_df["label"].value_counts().sort_index())


=== Original train/val sizes ===
Train: (254159, 2) Validation: (8101, 2)
Train label dist:
 label
0     90403
1     62297
2    101459
Name: count, dtype: int64

=== Balanced train ===
Balanced Train: (186891, 2)
Label dist:
 label
0    62297
1    62297
2    62297
Name: count, dtype: int64

Validation label dist (kept original):
 label
0    2740
1    2098
2    3263
Name: count, dtype: int64


/tmp/ipykernel_1497212/3243836472.py:31: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return g.apply(lambda x: x.sample(n=n, replace=False, random_state=random_state)).reset_index(drop=True)


In [3]:
# =========================
# 2) Tokenization & datasets
# =========================
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer

ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df)
})
num_labels = 3  # 0=neg, 1=neu, 2=pos

tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tok.padding_side = "right"
tok.truncation_side = "right"
if tok.pad_token_id is None and tok.eos_token_id is not None:
    tok.pad_token = tok.eos_token

MAX_LEN = 128
def tokenize_batch(batch):
    enc = tok(batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN)
    enc["labels"] = batch["label"]
    return enc

tok_ds = ds.map(tokenize_batch, batched=True, remove_columns=["text","label"])
tok_ds = tok_ds.with_format("torch")

/home/exx/venvs/llamaFT_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 8101/8101 [00:00<00:00, 23818.78 examples/s]


In [4]:
# =========================
# 3) Model (bf16/fp16) + LoRA
# =========================
from transformers import AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

use_bf16    = torch.cuda.is_bf16_supported()
torch_dtype = torch.bfloat16 if use_bf16 else torch.float16

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    problem_type="single_label_classification",
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
)
model.to("cuda")

if tok.pad_token_id is None and tok.eos_token_id is not None:
    tok.pad_token = tok.eos_token
model.config.pad_token_id = tok.pad_token_id
model.config.use_cache = False
model.gradient_checkpointing_enable()

# Helpful label mapping for downstream
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {v:k for k,v in id2label.items()}
model.config.id2label = id2label
model.config.label2id = label2id

# LoRA config
lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj", "up_proj", "down_proj"
        ],  
)
model = get_peft_model(model, lora_cfg)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTrainable params (LoRA + head): {trainable/1e6:.2f}M")


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 15.92it/s]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.1-8B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Trainable params (LoRA + head): 41.96M


In [5]:
# =========================
# 4) Train
# =========================
from transformers import TrainingArguments, Trainer, EvalPrediction
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=-1)
    acc = accuracy_score(p.label_ids, preds)
    macro_f1 = f1_score(p.label_ids, preds, average="macro")
    return {"accuracy": float(acc), "macro_f1": float(macro_f1)}

BATCH_SIZE = 8
GRAD_ACCUM = 4

args = TrainingArguments(
    output_dir=str(OUT_DIR),
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    num_train_epochs=6,
    warmup_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    logging_steps=50,
    bf16=use_bf16,
    fp16=not use_bf16,
    dataloader_pin_memory=True,
    dataloader_num_workers=8,
    optim="adamw_torch",
    report_to="none",
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tok_ds["train"],
    eval_dataset=tok_ds["validation"],
    tokenizer=tok,
    compute_metrics=compute_metrics,
)

train_result = trainer.train()
final_eval   = trainer.evaluate()
print("\n=== Eval Result ===\n", final_eval)

/home/exx/venvs/llamaFT_env/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/35040 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after 

{'loss': 2.7383, 'grad_norm': 18.829296112060547, 'learning_rate': 1e-05, 'epoch': 0.01}


  0%|          | 100/35040 [02:58<17:34:23,  1.81s/it]

{'loss': 1.1691, 'grad_norm': 17.597705841064453, 'learning_rate': 2e-05, 'epoch': 0.02}


  0%|          | 150/35040 [04:27<17:18:43,  1.79s/it]

...

{'eval_loss': 1.5248522758483887, 'eval_accuracy': 0.7747191704727812, 'eval_macro_f1': 0.7620281364451916, 'eval_runtime': 134.9184, 'eval_samples_per_second': 60.044, 'eval_steps_per_second': 7.508, 'epoch': 6.0}


100%|██████████| 35040/35040 [18:01:12<00:00,  1.85s/it]

{'train_runtime': 64872.3239, 'train_samples_per_second': 17.285, 'train_steps_per_second': 0.54, 'train_loss': 0.2951020560444218, 'epoch': 6.0}



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The


=== Eval Result ===
 {'eval_loss': 0.47681427001953125, 'eval_accuracy': 0.7954573509443279, 'eval_macro_f1': 0.7848311034243377, 'eval_runtime': 134.1684, 'eval_samples_per_second': 60.379, 'eval_steps_per_second': 7.55, 'epoch': 5.999486345347145}


In [10]:
# =========================
# 5) Save: heads/adapters/configs + eval artifacts
# =========================
import torch
from sklearn.metrics import confusion_matrix, classification_report

# 5.1 Save LoRA adapter + tokenizer into stage1Result/output/adapter
ADAPTER_DIR = OUTPUT_DIR / "adapter"
ADAPTER_DIR.mkdir(parents=True, exist_ok=True)
model.save_pretrained(ADAPTER_DIR)     # includes LoRA weights & peft config
tok.save_pretrained(ADAPTER_DIR)

# 5.2 Save the 3-way classification head
head = getattr(model, "score", None) or getattr(model, "classifier", None)
assert head is not None, "Could not find classification head (score/classifier)."
torch.save(head.state_dict(), OUTPUT_DIR / "sent_head.pt")

# 5.3 Save configs & training args
# - LoRA config

import json

def to_jsonable(obj):
    if isinstance(obj, set):
        return sorted(list(obj))
    if isinstance(obj, dict):
        return {k: to_jsonable(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [to_jsonable(x) for x in obj]
    return obj

lora_cfg_dict = to_jsonable(lora_cfg.to_dict())
with open(OUTPUT_DIR / "lora_config.json", "w") as f:
    json.dump(lora_cfg_dict, f, indent=2)

# with open(OUTPUT_DIR / "lora_config.json", "w") as f:
#     json.dump(lora_cfg.to_dict(), f, indent=2)


# - HF model config
model.config.to_json_file(OUTPUT_DIR / "hf_model_config.json")

# - TrainingArguments
# Save TrainingArguments to JSON (string -> file)
with open(OUTPUT_DIR / "training_args.json", "w") as f:
    f.write(args.to_json_string())
# args.to_json_file(OUTPUT_DIR / "training_args.json")

# - Dataset sizes & seed
with open(OUTPUT_DIR / "run_meta.json", "w") as f:
    json.dump({
        "seed": SEED,
        "train_size": int(len(train_df)),
        "val_size": int(len(val_df)),
        "balanced_train_label_dist": train_df["label"].value_counts().sort_index().to_dict(),
        "val_label_dist": val_df["label"].value_counts().sort_index().to_dict(),
        "id2label": id2label
    }, f, indent=2)

# 5.4 Save evaluation metrics
# Basic metrics from trainer.evaluate()
with open(EVAL_DIR / "metrics_eval.json", "w") as f:
    json.dump(final_eval, f, indent=2)

# Detailed predictions on validation set
pred = trainer.predict(tok_ds["validation"])
logits = pred.predictions
y_true = pred.label_ids
y_pred = np.argmax(logits, axis=-1)

# Confusion matrix / report
cm = confusion_matrix(y_true, y_pred, labels=[0,1,2])
cm_df = pd.DataFrame(cm, index=[id2label[i] for i in [0,1,2]],
                        columns=[id2label[i] for i in [0,1,2]])
cm_df.to_csv(EVAL_DIR / "confusion_matrix.csv", index=True)

report_str = classification_report(y_true, y_pred, target_names=[id2label[i] for i in [0,1,2]], digits=4)
with open(EVAL_DIR / "classification_report.txt", "w") as f:
    f.write(report_str)

# Per-sample validation outputs with original text for inspection
val_with_pred = val_df.copy()
val_with_pred["pred_label"] = y_pred
val_with_pred["pred_name"]  = val_with_pred["pred_label"].map(id2label)
val_with_pred["true_name"]  = val_with_pred["label"].map(id2label)
val_with_pred.to_csv(EVAL_DIR / "val_predictions_with_text.csv", index=False)

# Also save raw logits (optional, big but useful for analysis)
np.save(EVAL_DIR / "val_logits.npy", logits)

print("\n== Saved Stage-1 artifacts ==")
print("Adapter dir:", ADAPTER_DIR)
print("Sentiment head:", OUTPUT_DIR / "sent_head.pt")
print("LoRA config:", OUTPUT_DIR / "lora_config.json")
print("HF model config:", OUTPUT_DIR / "hf_model_config.json")
print("Training args:", OUTPUT_DIR / "training_args.json")
print("Eval metrics:", EVAL_DIR / "metrics_eval.json")
print("Confusion matrix CSV:", EVAL_DIR / "confusion_matrix.csv")
print("Classification report:", EVAL_DIR / "classification_report.txt")
print("Val predictions CSV:", EVAL_DIR / "val_predictions_with_text.csv")
print("Val logits NPY:", EVAL_DIR / "val_logits.npy")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 


== Saved Stage-1 artifacts ==
Adapter dir: /media/veracrypt1/DS5500/project/stage1/stage1Result/output/adapter
Sentiment head: /media/veracrypt1/DS5500/project/stage1/stage1Result/output/sent_head.pt
LoRA config: /media/veracrypt1/DS5500/project/stage1/stage1Result/output/lora_config.json
HF model config: /media/veracrypt1/DS5500/project/stage1/stage1Result/output/hf_model_config.json
Training args: /media/veracrypt1/DS5500/project/stage1/stage1Result/output/training_args.json
Eval metrics: /media/veracrypt1/DS5500/project/stage1/stage1Result/eval/metrics_eval.json
Confusion matrix CSV: /media/veracrypt1/DS5500/project/stage1/stage1Result/eval/confusion_matrix.csv
Classification report: /media/veracrypt1/DS5500/project/stage1/stage1Result/eval/classification_report.txt
Val predictions CSV: /media/veracrypt1/DS5500/project/stage1/stage1Result/eval/val_predictions_with_text.csv
Val logits NPY: /media/veracrypt1/DS5500/project/stage1/stage1Result/eval/val_logits.npy


# END

In [1]:
# Print Stage-1 metrics and confusion matrix from saved artifacts
from pathlib import Path
import json, pandas as pd

EVAL_DIR = Path("/Users/penglishao/Desktop/DS5500/project/stage1/stage1Result/eval")

# Accuracy / Macro F1
with open(EVAL_DIR / "metrics_eval.json", "r") as f:
    metrics = json.load(f)
acc = metrics.get("eval_accuracy", metrics.get("accuracy"))
macro_f1 = metrics.get("eval_macro_f1", metrics.get("macro_f1"))
print(f"Accuracy: {acc:.4f}")
print(f"Macro F1: {macro_f1:.4f}")

# Confusion matrix
cm_df = pd.read_csv(EVAL_DIR / "confusion_matrix.csv", index_col=0)
print("\nConfusion Matrix (rows=true, cols=pred):")
print(cm_df.to_string())

# (Optional) Full classification report
try:
    print("\nClassification report:\n")
    print((EVAL_DIR / "classification_report.txt").read_text())
except Exception:
    pass


Accuracy: 0.7955
Macro F1: 0.7848

Confusion Matrix (rows=true, cols=pred):
          negative  neutral  positive
negative      2273      444        23
neutral        387     1497       214
positive        36      553      2674

Classification report:

              precision    recall  f1-score   support

    negative     0.8431    0.8296    0.8363      2740
     neutral     0.6002    0.7135    0.6520      2098
    positive     0.9186    0.8195    0.8662      3263

    accuracy                         0.7955      8101
   macro avg     0.7873    0.7875    0.7848      8101
weighted avg     0.8106    0.7955    0.8006      8101

